## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,55.00,84,88,5.99,Clouds
1,1,Jalalabad,AF,34.4265,70.4515,70.61,29,0,3.31,Clear
2,2,Busselton,AU,-33.6500,115.3333,52.00,85,40,7.47,Clouds
3,3,Athabasca,CA,54.7169,-113.2854,41.00,56,94,11.50,Clouds
4,4,Sumenep,ID,-7.0167,113.8667,77.58,81,89,4.94,Clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 75


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,55.00,84,88,5.99,Clouds
1,1,Jalalabad,AF,34.4265,70.4515,70.61,29,0,3.31,Clear
2,2,Busselton,AU,-33.6500,115.3333,52.00,85,40,7.47,Clouds
3,3,Athabasca,CA,54.7169,-113.2854,41.00,56,94,11.50,Clouds
6,6,Chitungwiza,ZW,-18.0127,31.0756,64.00,68,68,5.01,Clouds
7,7,Bluff,NZ,-46.6000,168.3333,55.00,84,77,5.99,Clouds
10,10,Bengkulu,ID,-3.8004,102.2655,74.35,91,82,4.63,Clouds
13,13,Hobart,AU,-42.8794,147.3294,55.40,58,82,13.80,Clouds
15,15,Himatangi,NZ,-40.4000,175.3167,55.00,86,69,1.99,Clouds
16,16,Lasa,IT,46.6166,10.7002,59.00,45,20,6.91,Clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                360
City                   360
Country                358
Lat                    360
Lng                    360
Max Temp               360
Humidity               360
Cloudiness             360
Wind Speed             360
Current Description    360
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how = 'all')
clean_df = preferred_cities_df

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,55.00,Clouds,-46.1927,168.8643,
1,Jalalabad,AF,70.61,Clear,34.4265,70.4515,
2,Busselton,AU,52.00,Clouds,-33.6500,115.3333,
3,Athabasca,CA,41.00,Clouds,54.7169,-113.2854,
6,Chitungwiza,ZW,64.00,Clouds,-18.0127,31.0756,
7,Bluff,NZ,55.00,Clouds,-46.6000,168.3333,
10,Bengkulu,ID,74.35,Clouds,-3.8004,102.2655,
13,Hobart,AU,55.40,Clouds,-42.8794,147.3294,
15,Himatangi,NZ,55.00,Clouds,-40.4000,175.3167,
16,Lasa,IT,59.00,Clouds,46.6166,10.7002,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna(how = 'all')
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,55.00,Clouds,-46.1927,168.8643,Ellie's Villa
1,Jalalabad,AF,70.61,Clear,34.4265,70.4515,انصاف چهاردهي هوټل
2,Busselton,AU,52.00,Clouds,-33.6500,115.3333,Observatory Guest House
3,Athabasca,CA,41.00,Clouds,54.7169,-113.2854,Athabasca Lodge Motel
6,Chitungwiza,ZW,64.00,Clouds,-18.0127,31.0756,Damview Hotel


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "Weatherpy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))